In [3]:
#Regression>>>predict the output of a continuous value
#Dataset>>>classic Auto MPG 
!pip install tensorflow --upgrade

# Use seaborn for pairplot
!pip install seaborn

# Use some functions from tensorflow_docs
!pip install git+https://github.com/tensorflow/docs

from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

print(tf.__version__)
name_db = "auto-mpg.data"
address_db = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"

#Downloading the dataset
dataset_path = keras.utils.get_file(name_db, address_db)
dataset_path



#Importing the data using Pandas library
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names, na_values = "?", comment='\t', sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()
dataset.tail()




Requirement already up-to-date: tensorflow in c:\users\asob\anaconda3\lib\site-packages (2.4.1)
  Cloning https://github.com/tensorflow/docs to c:\users\asob\appdata\local\temp\pip-req-build-cs8sdyw4


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/tensorflow/docs 'C:\Users\asob\AppData\Local\Temp\pip-req-build-cs8sdyw4'
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?
